# Why this Competition?
This competition provides an unique oppertunity for Data Science beginners to participate in a Hackathon style challenge. It also provides the unique oppertunities for beginners to get their hands dirty and indulge is practical application of ML and do one of the basic tasks machine learning algorithms are capable of doing:- Classification.

This competition has the right mix to Catergorical and Numerical features we might expect in a practical problem and this helps us know how to leverage both of thhem in conjugation for a Classification task.

# Problem Statement
The goal of this competition is to provide a fun, and approachable for anyone, tabular dataset. These competition will be great for people looking for something in between the Titanic Getting Started competition and a Featured competition.

The dataset is used for this competition is synthetic but based on a real dataset and generated using a CTGAN.

So we are sort of dealing with a variation of actual real-world data and here as Data Scientists are expected to predict the Multi-Class Classification based on these features.

## Expected Outcome:-
* Build a model to predict the category on an eCommerce product given various attributes about the listing.
* Grading Metric: Multi-Class Log Loss

## Problem Category:-
From the data and objective its is evident that this is a Multi-Class Classification Problem in the Tabular Data format.

So without further ado, let's now start with some basic imports to take us through this journey:-

In [ ]:
! pip install --quiet pytorch-tabnet

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os

# Visialisation
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

# Machine Learning

# Utils
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_validate
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn import preprocessing

#Feature Selection
from sklearn.feature_selection import chi2, f_classif, f_regression
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold

# Metrics
from sklearn.metrics import accuracy_score, log_loss

# Models
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
data_dir = '../input/tabular-playground-series-may-2021'

train_file_path = os.path.join(data_dir, 'train.csv')
test_file_path = os.path.join(data_dir, 'test.csv')
sample_sub_file_path = os.path.join(data_dir, 'sample_submission.csv')

print(f'Train file: {train_file_path}')
print(f'Train file: {test_file_path}')
print(f'Train file: {sample_sub_file_path}')

In [ ]:
RANDOM_SEED = 42

In [ ]:
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything()

In [ ]:
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
sub_df = pd.read_csv(sample_sub_file_path)

# Target Encoding

In [ ]:
mapping_dict = {'Class_1':1, 'Class_2':2, 'Class_3':3, 'Class_4':4}
train_df['target'] = train_df['target'].map(mapping_dict)

# Feature Scaling

In [ ]:
not_features = ['id', 'target']
features = []
for feat in train_df.columns:
    if feat not in not_features:
        features.append(feat)
print(features)

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(train_df[features])
train_df[features] = scaler.transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

# KFold Splits
Before we move on to feature engineering, it is always a good idea to perform cross validation splits. In that way, we will not risk any data leakage and would be more certain of the validation set being aptly represenative of the real world unknown data.

In [ ]:
NUM_SPLITS = 5

train_df["kfold"] = -1
train_df = train_df.sample(frac=1).reset_index(drop=True)
y = train_df.target.values
kf = StratifiedKFold(n_splits=NUM_SPLITS)
for f, (t_, v_) in enumerate(kf.split(X=train_df, y=y)):
    train_df.loc[v_, 'kfold'] = f
    
train_df.head()

# Features Selection
We need to select only the important features for better performance of the model. As unnecessary in best case scenario will not add to any productive calculation of the algorithm or in worst case scenario 'confuse' the model.

To do the same let's create a wrapper class that has all the built in statistical tests required to perform feature selection and takes some basic inputs from user and spits out the required features.

In [ ]:
# From https://github.com/abhishekkrthakur/approachingalmost
class UnivariateFeatureSelction:
    def __init__(self, n_features, problem_type, scoring, return_cols=True):
        """
        Custom univariate feature selection wrapper on
        different univariate feature selection models from
        scikit-learn.
        :param n_features: SelectPercentile if float else SelectKBest
        :param problem_type: classification or regression
        :param scoring: scoring function, string
        """
        self.n_features = n_features
        
        if problem_type == "classification":
            valid_scoring = {
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function")
            
        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features
            )
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile=int(n_features * 100)
            )
        else:
            raise Exception("Invalid type of feature")
    
    def fit(self, X, y):
        return self.selection.fit(X, y)
    
    def transform(self, X):
        return self.selection.transform(X)
    
    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)
    
    def return_cols(self, X):
        if isinstance(self.n_features, int):
            mask = SelectKBest.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
                    
        elif isinstance(self.n_features, float):
            mask = SelectPercentile.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
        else:
            raise Exception("Invalid type of feature")
        
        return selected_features

In [ ]:
ufs = UnivariateFeatureSelction(
    n_features=1.0,
    problem_type="classification",
    scoring="f_classif"
)

ufs.fit(train_df[features], train_df['target'].values.ravel())
selected_features = ufs.return_cols(train_df[features])

I have already shared a baseline notebook [here](https://www.kaggle.com/manabendrarout/soft-voting-ensemble-starter-tps-may21/comments#1290740) if you want to understand the data. For the sake of not repeating myself, let's get started directly with tabnet here...

# What is Tabnet?
![Tabnet Google](https://warehouse-camo.ingress.cmh1.psfhosted.org/88a031698031ed731cf65dc79a2f0085dbce4670/68747470733a2f2f6769746875622e636f6d2f74697475313939342f74662d5461624e65742f626c6f622f6d61737465722f696d616765732f5461624e65742e706e673f7261773d74727565 "Tabnet")

Tabnet is a novel high-performance and interpretable canonical deep tabular data learning architecture. TabNet uses sequential attention to choose which features to reason from at each decision step, enabling interpretability and more efficient learning as the learning capacity is used for the most salient features. TabNet outperforms other variants on a wide range of non-performance-saturated tabular datasets and yields interpretable feature attributions plus insights into its global behavior.

TabNet uses a sequential attention mechanism to choose a subset of semantically meaningful features to process at each decision step. Instance-wise feature selection enables efficient learning as the model capacity is fully used for the most salient features, and also yields more interpretable decision making via visualization of selection masks.

For deatils please refer [this](https://arxiv.org/pdf/1908.07442.pdf) awesome paper by Google.

TabNet has two useful implemnetations in [Pytorch](https://github.com/dreamquark-ai/tabnet) and, by the original authors, in [Tensorflow](https://github.com/google-research/google-research/tree/master/tabnet). In this notebook we will look at the Pytorch implementation which has more community support and benchmarking, and compare our results under default parameters to a Feed-forward Dense Neural Network provided in Scikit-learn, and CatBoost. We will compare the global feature importances described by TabNet and our Catboost and let readers get a sense of the usability and promise of these tools.

In [ ]:
def get_tabnet_model():
    classifier = TabNetClassifier(
      n_d=64, n_a=64, n_steps=5,
      # gamma=1.5, n_independent=2, n_shared=2,
      # lambda_sparse=1e-4, momentum=0.1,
      optimizer_fn=torch.optim.Adam,
      optimizer_params=dict(lr=2e-2),
      # scheduler_fn=torch.optim.lr_scheduler.StepLR,
      # scheduler_params = {"gamma": 0.95, "step_size": 20},
      verbose=1, seed=42)

    return classifier

In [ ]:
EPOCHS = 100

In [ ]:
best_clf = None
best_score = 10000
all_clfs = []

for k in range(NUM_SPLITS):
    print(f'******************* Staring fold: {k+1} *******************')
    current_fold = k
    train_fold = train_df[train_df['kfold'] != current_fold]
    valid_fold = train_df[train_df['kfold'] == current_fold]

    X_train = train_fold[selected_features].to_numpy()
    y_train = train_fold['target'].to_numpy().squeeze()

    X_valid = valid_fold[selected_features].to_numpy()
    y_valid = valid_fold['target'].to_numpy().squeeze()

    classifier = get_tabnet_model()

    classifier.fit(X_train, y_train,
                 eval_set=[(X_valid, y_valid)],
                 eval_name=['valid'],
                 patience=20,
                 max_epochs=EPOCHS,
                #  batch_size=1024,
                #  virtual_batch_size=256,
                 eval_metric=['logloss'])
    print(' ')
    preds = classifier.predict(X_valid)
    pred_prob = classifier.predict_proba(X_valid)
    print(f'Accuracy: {round(accuracy_score(y_valid, preds), 5)}')
    print(f'Log Loss: {round(log_loss(y_valid, pred_prob), 5)}')
    if round(log_loss(y_valid, pred_prob), 5) < best_score:
        best_score = round(log_loss(y_valid, pred_prob), 5)
        best_clf = classifier
    all_clfs.append(classifier)
    print('***********************************************************')
    print(' ')

In [ ]:
print(f'Best Classifier had score of {best_score}')

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(best_clf.history['valid_logloss']);
plt.xlim([0, 100])
plt.show();

# Attention Masks Visualization

In [ ]:
X_test = test_df[selected_features].to_numpy()
explain_matrix, masks = classifier.explain(X_test)

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20,20))

for i in range(5):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")

# Submission Prediction

In [ ]:
X_test = test_df[selected_features].to_numpy()
pred_prob = None
for k in range(NUM_SPLITS):
    clf = all_clfs[k]
    if pred_prob is None:
        pred_prob = clf.predict_proba(X_test)
    else:
        pred_prob += clf.predict_proba(X_test)

pred_prob /= NUM_SPLITS

submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['Class_1'] = pred_prob[:, 0]
submission['Class_2'] = pred_prob[:, 1]
submission['Class_3'] = pred_prob[:, 2]
submission['Class_4'] = pred_prob[:, 3]

In [ ]:
submission.head()

In [ ]:
submission.to_csv("tabnet_sub.csv",index=False)

**If you found this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels.** 😊